# **Лабораторная работа №3 по вычислительной математикe.**

## Работу выполнил: Шурыгин Антон, Б01 - 909.

### **IV. 12.2**

In [4]:
from scipy.optimize import fsolve
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp

In [12]:
x = sp.Symbol('x')
y = sp.Symbol('y')

eps = 10e-6

def norm(vec):
  return max(abs(vec))

## Рассмотрим систему нелинейных уравнений:

$$F(x) = 0, \:\: F(x), x \in R^{n}$$

In [8]:
eqs = {"f1"  : sp.tan(y - x) + y*x - 0.3, 
       "f2" : (y ** 2) + (x ** 2) - 1.5 }

for eq in eqs.items():
    print(f'{eq[0]} : {eq[1]}')

f1 : x*y - tan(x - y) - 0.3
f2 : x**2 + y**2 - 1.5


### Матрицей якоби для данной системы будет:
$$F'(x) = \frac{\partial F(x)}{x}$$

In [7]:
jcbn = {
             "f1_x" : sp.diff(eqs['f1'], x, 1),
             "f1_y" : sp.diff(eqs['f1'], y, 1),
             "f2_x" : sp.diff(eqs['f2'], x, 1),
             "f2_y" : sp.diff(eqs['f2'], y, 1)
       }


for jc_it in jcbn.items():
    print(f'{jc_it[0]} : {jc_it[1]}')

f1_x : y - tan(x - y)**2 - 1
f1_y : x + tan(x - y)**2 + 1
f2_x : 2*x
f2_y : 2*y


## **Реализация метода Ньютона**

 $$ x^{k+1} = x^{k} - [F'(x^{k})]^{-1}F(x^{k}) $$

## Сводим посталвенную задачу к решению СЛАУ
#### Пусть $\delta x^{k}= x^{k+1} - x^{k}$

 $$ \frac{\partial F(x^{k})}{\partial x} \cdot \delta x^{k} = -F(x^{k}) $$

### До тех пор, пока не достигнута точнсть $||x^{k + 1} - x^{k}|| < \varepsilon$ находим поправку $\delta x^{k}$ и переходм к следудющему приближению.



In [31]:
def jcbn_calc(jcbn, x_k):
    return  np.array([[float(jcbn['f1_x'].subs({x : x_k[0], y: x_k[1]})),
                       float(jcbn['f1_y'].subs({x : x_k[0], y: x_k[1]}))], 
                      [float(jcbn['f2_x'].subs({x : x_k[0], y: x_k[1]})),
                       float(jcbn['f2_y'].subs({x : x_k[0], y: x_k[1]}))]])


def f_calc(eqs, x_k):
    return np.array([float(-eqs["f1"].subs({x : x_k[0], y: x_k[1]})), 
                     float(-eqs["f2"].subs({x : x_k[0], y: x_k[1]}))])


def newton_meth(x_0):

    x_k = x_0
    greater = True

    while greater:

        f_k = f_calc(eqs, x_k)
        jcbn_k = jcbn_calc(jcbn, x_k)

        dx_k = np.linalg.solve(jcbn_k, f_k)
        x_k_1 = x_k + dx_k

        greater = np.linalg.norm(x_k_1 - x_k) > eps

        x_k = x_k_1

    return x_k_1

### Найдем решение методом Ньютона в окрестности точки (1, 1)

In [30]:
root = newton_meth([1, 1])
print(f'x={root[0]}, y={root[1]}')

x=1.0294058595953521, y=0.663568818066159


### Найдем решение методом Ньютона в окрестности точки (-1, -1)

In [29]:
root = newton_meth([-1, -1])
print(f'x={root[0]}, y={root[1]}')

x=-0.663568818066159, y=-1.0294058595953521


# Проверим корректность полученных значений с помощью метода ```fsolve```
### Решения, полученные с помощью модуля scipy

In [28]:
def solve_non_lin(vars):
    x, y = vars[0], vars[1]
    
    return [ np.tan(y - x) + y * x - 0.3,
             (y ** 2) + (x ** 2) - 1.5    ]

### Найдем решение в окрестности точки (1, 1)

In [27]:
root = fsolve(solve_non_lin, [1, 1])
print(f'x={root[0]}, y={root[1]}')

x=1.0294058595375626, y=0.6635688180963026


### Найдем решение в окрестности точки (-1, -1)

In [26]:
root = fsolve(solve_non_lin, [-1, -1])
print(f'x={root[0]}, y={root[1]}')

x=-0.6635688180963026, y=-1.0294058595375626
